In [3]:
# 企業の財務情報を取得してデータフレームに格納する関数を定義
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

def make_table(com_id):
    date = []
    var_dic = {}
    url = 'https://www.buffett-code.com/company/' + com_id + '/financial'
    html=requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")
    tables=soup.find_all("table",class_ = "table custom-table table-hover table-condensed")

    for i in tables[0].find('thead').find_all('th')[1:]:
        date.append(i.get_text().replace('年','/').replace('月期',''))
    
    var_dic['date'] = date
    
    for table in tables:
        make_df(table,var_dic)
        
    return pd.DataFrame(var_dic).convert_objects(convert_numeric=True).replace('-',np.nan)

def make_df(table,var_dic):
    for j in table.find('tbody').find_all('tr'):
        value = [value.get_text().replace(',','').replace('%','').replace('人','').replace('倍','').replace('株','').replace('ヶ月','').replace('日','') for value in j.find_all('td')]
        var_dic[value[0]] = value[1:]
    return var_dic

In [15]:
# idsに定義した証券コードの財務データをsqliteに格納をする
ids = ['8411','7203']
conn = sqlite3.connect('db.sqlite3')
for com_id in ids:
    make_table(com_id).to_sql('table_' + com_id,conn,if_exists='replace',index=None)
conn.close()

C:\Users\hirou\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Users\hirou\Anaconda3\lib\site-packages\pandas\core\generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


In [17]:
# 格納したデータを確認する
conn = sqlite3.connect('db.sqlite3')
df=pd.read_sql_query('SELECT * FROM table_7203', conn)
print(df)
conn.close()

     date       売上高    前期比        売上原価      売上総利益  売上総利益率        販管費     営業利益  \
0  2016/3  28403118  106.5  22600000.0  5800000.0    20.4  2940000.0  2850000   
1  2017/3  27597193   79.2  22700000.0  4860000.0    17.6  2870000.0  1990000   
2  2018/3  29400000  136.1  23900000.0  5490000.0    18.7  3090000.0  2400000   
3  2019/3  30000000   90.4         NaN        NaN     NaN        NaN  2550000   

   営業利益率   EBITDA   ...    のれん償却費       投資CF      財務CF      フリーCF      設備投資額  \
0   10.0  4479810   ...      None -3180000.0 -424000.0  1278320.0  4070000.0   
1    7.2  3600000   ...      None -2970000.0 -375000.0   440000.0  3530000.0   
2    8.2  4130000   ...      None -3660000.0 -449000.0   550000.0  1300000.0   
3    8.5  4280000   ...      None        NaN       NaN        NaN        NaN   

       研究開発費        発行済式総数      従業員数  売上/従業員  営利/従業員  
0  1060000.0  3.337997e+09  348877.0    81.0     8.0  
1  1040000.0  3.262997e+09  364445.0    75.0     5.0  
2  1060000.0  3.262997e+09  